## Dependencies

In [1]:
import pandas as pd
import numpy as np
import os
import shutil
from datetime import date
import csv
import time
import zipfile
import requests
import steam
from steam.steamid import SteamID
from steam.webapi import WebAPI, get
import steamleaderboards as sl
import os,sys,stat
from zipfile import ZipFile, ZipInfo

# 1. unpack stats from folder

(from zip)

In [2]:
# App will find directory on computer or user can input the directory of the stats folder
#dataDir = 'G:\Steam\steamapps\common\FPSAimTrainer\FPSAimTrainer\stats_comp.zip'
#target = ['Kills:', 'Avg TTK:', 'Score:', 'Scenario:']
#temp_folder = '../temp'
#
## unzip and extract to a temporary folder
#with zipfile.ZipFile(dataDir, 'r') as zip_ref:
#   zip_ref.extractall(temp_folder) # If files with same filename are already present at extraction location then it will overwrite those files.
#

In [3]:
#unzipped_stats = temp_folder + '\stats'
#
#try:
#    stats = pd.read_csv(unzipped_stats)
#except:
#    stats = pd.DataFrame(columns=['Kills', 'Avg TTK', 'Score', 'Scenario'])

FROM DIRECTORY

In [4]:
dataDir = 'G:\Steam\steamapps\common\FPSAimTrainer\FPSAimTrainer\stats'
target = ['Kills:', 'Avg TTK:', 'Score:', 'Scenario:']

try:
    stats = pd.read_csv(dataDir + '\\stats.csv')
except:
    stats = pd.DataFrame(columns=['Kills', 'Avg TTK', 'Score', 'Scenario'])

In [5]:
# Empty dict to store our csv data
data = {}

# Go through all the files and store our target data in our empty dictionary, including the datetime of the file.
for filename in os.listdir(dataDir):
    
    f = open(f'{dataDir}/{filename}', 'r')
    with f as csv_file:
        csv_reader = csv.reader(csv_file)
        for line in csv_reader:
            try:
                if line[0] in target:
                    data[(line[0][:-1])] = line[1]     
            except IndexError:
                continue
        f.close()

    scenario_date = f'{dataDir}/{filename}'[-29:-10]
    data["Date"] = scenario_date
    df = pd.DataFrame(data, index=[0])
    stats = pd.concat([stats,df]).drop_duplicates().reset_index(drop=True)

# CLEAN


In [6]:
stats.head(3)

,Kills,Avg TTK,Score,Scenario,Date
0,2,0,10792.587891,180 Regen Warzone,2021.07.27-22.47.26
1,12,0,22690.832031,180 Regen Warzone,2021.07.27-22.48.41
2,7,0,15467.339844,180 Regen Warzone,2021.07.27-22.49.53


In [7]:
stats.isna().sum() #NO Nan

Kills       0
Avg TTK     0
Score       0
Scenario    0
Date        0
dtype: int64

In [8]:
stats.duplicated().sum() #NO duplicates

0

In [9]:
stats['Kills'] = pd.to_numeric(stats['Kills'], downcast='integer')
stats['Avg TTK'] = pd.to_numeric(stats['Avg TTK'], downcast='float')
stats['Score'] = pd.to_numeric(stats['Score'], downcast='float')
stats['Date'] = pd.to_datetime(stats['Date'], yearfirst=True, format=('%Y.%m.%d-%H.%M.%S'),exact=False)

In [10]:
stats.head()

,Kills,Avg TTK,Score,Scenario,Date
0,2,0.0,10792.587891,180 Regen Warzone,2021-07-27 22:47:26
1,12,0.0,22690.832031,180 Regen Warzone,2021-07-27 22:48:41
2,7,0.0,15467.339844,180 Regen Warzone,2021-07-27 22:49:53
3,9,0.0,16614.562500,180 Regen Warzone,2021-07-28 22:45:19
4,10,0.0,16942.285156,180 Regen Warzone,2021-07-28 22:46:26


# SAVE

In [11]:
stats.to_csv("../data/stats.csv", index=False)

# 2. CREATE COUNTER THAT COUNTS UP MY HIGHEST SCORING GAME IN REAL-TIME

### Search Bar

In [12]:
import difflib
scenario_names = (stats['Scenario'].unique()).astype(str)
user_input = 'patts voltaic silver' #tkinter search bar
closest_match = (difflib.get_close_matches(str(user_input), scenario_names, n=1,cutoff=0.45).pop(0))
closest_match

'patTS Voltaic Silver'

In [13]:
scenario = stats.loc[stats['Scenario'] == closest_match]
scenario

,Kills,Avg TTK,Score,Scenario,Date
1849,83,0.734935,83.00,patTS Voltaic Silver,2022-05-26 19:53:31
1850,84,0.726186,84.00,patTS Voltaic Silver,2022-05-26 19:54:46
1851,85,0.717643,85.00,patTS Voltaic Silver,2022-05-26 19:55:52
1852,85,0.717643,85.00,patTS Voltaic Silver,2022-05-26 19:56:59
1853,84,0.726186,84.50,patTS Voltaic Silver,2022-05-26 19:58:08
1854,77,0.792203,77.00,patTS Voltaic Silver,2022-05-29 15:00:42
1855,79,0.772147,79.00,patTS Voltaic Silver,2022-05-29 15:01:47
1856,79,0.772147,79.75,patTS Voltaic Silver,2022-05-29 15:02:51
1857,80,0.762496,80.00,patTS Voltaic Silver,2022-05-29 15:04:27
1858,78,0.782047,78.50,patTS Voltaic Silver,2022-05-29 15:05:32


### Date of highest score

In [14]:
import re

high_score_date = []

if sum(scenario['Score']) == 0.0:
    high_score_date.append(scenario.sort_values('Kills')['Date'].iloc[-1])
elif sum(scenario['Kills']) > 0 and sum(scenario['Score']) > 0.0:
    high_score_date.append(scenario.sort_values('Score')['Date'].iloc[-1])

hs_date = str(high_score_date[0])
hs_date

'2022-05-26 19:55:52'

### Now we search for the .csv file with the highest score

In [15]:
hs_year = hs_date[:4]
hs_month = hs_date[5:7]
hs_day = hs_date[8:10]
hs_hour = hs_date[11:13]
hs_min = hs_date[14:16]
hs_sec = hs_date[17:19]

In [20]:
import re
import time

Kill = []
Time = []
Acc = []
Challenge_start = []

for filename in os.listdir(dataDir):
    if f'{hs_year}.{hs_month}.{hs_day}-{hs_hour}.{hs_min}.{hs_sec} Stats.csv' in filename:
        f = open(f'{dataDir}/{filename}', 'r')
        with f as csv_file:
            csv_reader = csv.reader(csv_file)
            for line in csv_reader:
                try:
                    Kill.append(line[0])
                    Time.append(line[1])
                    Acc.append(line[7])
                except Exception:
                    break 
            for line in csv_reader:
                try:
                    if line[0] in 'Challenge Start:' and line[1] not in 'H:M:S.s':
                        Challenge_start.append(line[1])
                    else:
                        start = Time[0] - (Time[-1]-Time[0]) # deduce length between chalenge start and first kill
                        Challenge_start.append(start)
                except Exception:
                    continue              
        f.close()
Challenge_start

[]

In [ ]:
data_tuples = list(zip(Kill,Time,Acc))

high_score_df = (pd.DataFrame(data_tuples,columns=('Kill #','Time', 'Accuracy'))
                    .replace(['Kill #','Accuracy'], 0)
                    ).replace('Timestamp',''.join(Challenge_start))

high_score_df.head()

,Kill #,Time,Accuracy
0,0,18:01:32.082,0
1,1,18:01:33.001,1
2,2,18:01:34.055,0.5
3,3,18:01:35.103,1
4,4,18:01:35.755,1


CLEAN (i need the dates column type to be float in order to make my calculations)

In [ ]:
high_score_df['Kill #'] = pd.to_numeric(high_score_df['Kill #'], downcast='integer')
high_score_df['Accuracy'] = pd.to_numeric(high_score_df['Accuracy'], downcast='float')
high_score_df['Time'] = pd.to_datetime(high_score_df['Time'],format=('%H:%M:%S.%f'),exact=False)
high_score_df['ms'] = (high_score_df['Time'] - high_score_df['Time'].iloc[0]) / pd.to_timedelta('1ms')
high_score_df

,Kill #,Time,Accuracy,ms
0,0,1900-01-01 18:01:32.082,0.0,0.0
1,1,1900-01-01 18:01:33.001,1.0,919.0
2,2,1900-01-01 18:01:34.055,0.5,1973.0
3,3,1900-01-01 18:01:35.103,1.0,3021.0
4,4,1900-01-01 18:01:35.755,1.0,3673.0
...,...,...,...,...
948,948,1900-01-01 18:11:28.245,1.0,596163.0
949,949,1900-01-01 18:11:29.613,0.5,597531.0
950,950,1900-01-01 18:11:30.081,1.0,597999.0
951,951,1900-01-01 18:11:30.926,1.0,598844.0


## create counter.

In [ ]:
score_kills = 0

for index, value in enumerate(high_score_df['ms']):
    if value == 0.0:
        print('GO')      
        time.sleep((high_score_df['ms'][1])/1000)
        score_kills += 1
        print(f'Score/Kills: {score_kills}')
    else:
        time.sleep((high_score_df['ms'][index+1] - high_score_df['ms'][index])/1000)
        score_kills += 1
        print(f'Score/Kills: {score_kills}')
print('done')

GO
Score/Kills: 1
Score/Kills: 2
Score/Kills: 3
Score/Kills: 4
Score/Kills: 5
Score/Kills: 6
Score/Kills: 7
Score/Kills: 8
Score/Kills: 9
Score/Kills: 10
Score/Kills: 11
Score/Kills: 12
Score/Kills: 13
Score/Kills: 14


KeyboardInterrupt: 